In [1]:
import mpp_classifiers as mpp
import load_ship_data as lsd
import numpy as np
import time
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

Get data using our own data loader

NOTE: Confusion Matrix is incorrect. 

I corrected it in reported results, but left it this way in code

so that results would line up with report.

Rerunning with correct matrix would cause our random split to alter the results slightly

In [2]:

data_path="data/shipsnet.json"
train,test,valid=lsd.load_data_train_test_split(data_path)
def conf(pred,y):
    T0=sum([1 if x==y and y==0 else 0 for (x,y) in zip(pred,y)])
    T1=sum([1 if x==y and y==1 else 0 for (x,y) in zip(pred,y)])
    F0=sum([1 if x!=y and y==0 else 0 for (x,y) in zip(pred,y)])
    F1=sum([1 if x!=y and y==1 else 0 for (x,y) in zip(pred,y)])
    return np.array([[T0,F0],[F1,T1]])

Training Set Data Length:  2800   Label Length:  2800
TestingSet Set Data Length:  600  Label Length:  600
Validation Set Data Length:  600  Label Length:  600


Setup data splits and priors

In [3]:
ship_prob=np.sum(train[1])*1.0/train[1].shape[0]
p=np.array([1-ship_prob,ship_prob])
print(p)
Xtrain=train[0]
ytrain=train[1]
Xtest=test[0]
ytest=test[1]
Xvalid=valid[0]
yvalid=valid[1]
Xtrain=Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1]*Xtrain.shape[2]*Xtrain.shape[3])
Xtest=Xtest.reshape(Xtest.shape[0],Xtest.shape[1]*Xtest.shape[2]*Xtest.shape[3])
Xvalid=Xvalid.reshape(Xvalid.shape[0],Xvalid.shape[1]*Xvalid.shape[2]*Xvalid.shape[3])
print(Xtrain.shape)
print(Xtest.shape)
print(Xvalid.shape)

[0.74892857 0.25107143]
(2800, 19200)
(600, 19200)
(600, 19200)


Case 1. No PCA, even priors

In [4]:

p55=np.array([.5,.5])
t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p55)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.6766666666666666;
Classwise accuracy = [0.70769231 0.57931034];
The learning process takes 14.33341121673584 seconds.
[[322 133]
 [ 61  84]]


Case 2. No PCA, prior assumes ships are rare

In [5]:
p91=np.array([.9,.1])
t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p91)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.7583333333333333;
Classwise accuracy = [1. 0.];
The learning process takes 14.029608488082886 seconds.
[[455   0]
 [145   0]]


Case 1. No PCA, priors from training data

In [6]:

t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.76;
Classwise accuracy = [1.         0.00689655];
The learning process takes 14.041727066040039 seconds.
[[455   0]
 [144   1]]


PCA = 800 dimensions kept

In [7]:
pca = PCA(n_components=800)
pca.fit(Xtrain)
pca_Xtrain=pca.transform(Xtrain)
pca_Xtest=pca.transform(Xtest)
explained=np.sum(pca.explained_variance_ratio_)
print(explained)

0.99073861375588


Case 1. PCA=800, training priors used from now on

In [8]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.7616666666666667;
Classwise accuracy = [1.        0.0137931];
The learning process takes 0.06932425498962402 seconds.
[[455   0]
 [143   2]]


Case 2. PCA=800

In [9]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 2, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.9216666666666666;
Classwise accuracy = [0.95824176 0.80689655];
The learning process takes 24.75102972984314 seconds.
[[436  19]
 [ 28 117]]


Case 3. PCA=800

In [10]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 3, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.44166666666666665;
Classwise accuracy = [0.38901099 0.60689655];
The learning process takes 32.077874422073364 seconds.
[[177 278]
 [ 57  88]]


PCA =200 dimensions kept

In [11]:
pca2 = PCA(n_components=200)
pca2.fit(Xtrain)
pca2_Xtrain=pca2.transform(Xtrain)
pca2_Xtest=pca2.transform(Xtest)
explained=np.sum(pca2.explained_variance_ratio_)
print(explained)

0.9433241744622998


Case 1. PCA=200

In [12]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.7633333333333333;
Classwise accuracy = [1.         0.02068966];
The learning process takes 0.024446725845336914 seconds.
[[455   0]
 [142   3]]


Case 2. PCA=200

In [13]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 2, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.93;
Classwise accuracy = [0.98021978 0.77241379];
The learning process takes 0.53659987449646 seconds.
[[446   9]
 [ 33 112]]


Case 3. PCA=200

In [14]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 3, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.7933333333333333;
Classwise accuracy = [0.80879121 0.74482759];
The learning process takes 0.8946073055267334 seconds.
[[368  87]
 [ 37 108]]


Join training and validation since upcoming cross-validation will do this split for us

In [15]:
pca2 = PCA(n_components=200)
Xtrain=np.concatenate((Xtrain,Xvalid))
ytrain=np.concatenate((ytrain,yvalid))
pca2.fit(Xtrain)
pca2_Xtrain=pca2.transform(Xtrain)
pca2_Xtest=pca2.transform(Xtest)
explained=np.sum(pca2.explained_variance_ratio_)
print(explained)

0.9406837947054734


Setup for Kfold cross-validation

In [16]:
p=np.array([1-ship_prob,ship_prob])
X=pca2_Xtrain
y=ytrain
k=10

Case 1

In [17]:

kf=KFold(n_splits=k,random_state=None)
acc_scores=[]
for train_index,test_index in kf.split(X):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    y_model = mpp.mpp(X_train, y_train, X_test, 1, p)
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    acc_scores.append(acc_overall)
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))
y_model = mpp.mpp(X_train, y_train, X_test, 1, p)
acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
print(acc_overall)

Accuracy from each fold =  [0.7411764705882353, 0.7588235294117647, 0.7794117647058824, 0.7323529411764705, 0.7411764705882353, 0.7382352941176471, 0.7676470588235295, 0.7588235294117647, 0.7382352941176471, 0.7441176470588236]
Average Accuracy = 0.75
0.7441176470588236


Case 2

In [18]:
kf=KFold(n_splits=k,random_state=None)
acc_scores=[]
for train_index,test_index in kf.split(X):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    y_model = mpp.mpp(X_train, y_train, X_test, 2, p)
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    acc_scores.append(acc_overall)
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))
y_model = mpp.mpp(X_train, y_train, X_test, 2, p)
acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
print(acc_overall)



Accuracy from each fold =  [0.9235294117647059, 0.9088235294117647, 0.9294117647058824, 0.9411764705882353, 0.9235294117647059, 0.9235294117647059, 0.9058823529411765, 0.9205882352941176, 0.9264705882352942, 0.9352941176470588]
Average Accuracy = 0.9238235294117649
0.9352941176470588


Case 3

In [19]:
kf=KFold(n_splits=k,random_state=None)
acc_scores=[]
for train_index,test_index in kf.split(X):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    y_model = mpp.mpp(X_train, y_train, X_test, 3, p)
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    acc_scores.append(acc_overall)
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))
y_model = mpp.mpp(X_train, y_train, X_test, 3, p)
acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
print(acc_overall)



Accuracy from each fold =  [0.8, 0.8323529411764706, 0.8294117647058824, 0.8088235294117647, 0.8176470588235294, 0.788235294117647, 0.7764705882352941, 0.7823529411764706, 0.788235294117647, 0.7823529411764706]
Average Accuracy = 0.8005882352941176
0.7823529411764706
